# Programming Language Interoperability (Interop)

## Python

In [1]:
using PythonCall

In [2]:
@pyeval "3 + 3"

Python int: 6

In [4]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/roscar/work/github.com/RobertRosca/julia-hpc-workshop/.CondaPkg/env/lib/python3.11/site-packages/numpy/__init__.py'>

In [5]:
np.linalg.eigvals(np.random.rand(5, 5))

Python ndarray:
array([ 2.62144673+0.j        ,  0.53918828+0.j        ,
       -0.15233336+0.34654066j, -0.15233336-0.34654066j,
       -0.41759827+0.j        ])

In [6]:
M = rand(5, 5)

5×5 Matrix{Float64}:
 0.683177  0.70782   0.411005  0.582225  0.775679
 0.792823  0.939811  0.415071  0.47986   0.357398
 0.245343  0.326745  0.961803  0.128129  0.703248
 0.31821   0.7687    0.411612  0.551322  0.527867
 0.602407  0.436685  0.195505  0.081992  0.0861006

In [7]:
np.linalg.eigvals(M)

Python ndarray:
array([ 2.54641143+0.j        , -0.11916338+0.02345213j,
       -0.11916338-0.02345213j,  0.26427421+0.j        ,
        0.64985442+0.j        ])

In [9]:
@pyexec """
global sinpi, np
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [10]:
py_sinpi(x) = pyconvert(Float64, @pyeval("sinpi")(x))

py_sinpi (generic function with 1 method)

In [11]:
py_sinpi(10)

-1.2246467991473533e-15

In [12]:
using BenchmarkTools
@btime py_sinpi(10);
@btime sinpi(10); # built-in Julia function

  1.327 μs (3 allocations: 48 bytes)
  1.262 ns (0 allocations: 0 bytes)


## C

In [13]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

Compile to a shared library by piping `c_code` to gcc:

In [14]:
using Libdl
const Clib = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [15]:
Clib

"/tmp/jl_VOvCJcfcds.so"

Binding the function from the shared library:

In [16]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [17]:
c_sum(rand(10))

3.941699267800729

In [18]:
x = rand(10)
@btime c_sum($x);

  5.530 ns (0 allocations: 0 bytes)


## Mixing Julia, Python, and C

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [19]:
x = rand(10);

In [20]:
abs(py_sinpi(c_sum(x)))

0.18827630984552232

In [21]:
@btime abs(py_sinpi(c_sum($x)));

  1.305 μs (3 allocations: 48 bytes)


See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).

# Julia Microbenchmark: Summation

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [24]:
x = rand(10^7);

In [25]:
sum(x)

5.002258869510192e6

In [26]:
d = Dict() # to store the measurement results

Dict{Any, Any}()

## Python

In [27]:
using BenchmarkTools
using PythonCall

### numpy

In [28]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/roscar/work/github.com/RobertRosca/julia-hpc-workshop/.CondaPkg/env/lib/python3.11/site-packages/numpy/__init__.py'>

In [29]:
numpy_sum = np.sum

Python function: <function sum at 0x7f4756175620>

In [30]:
b = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 882 samples with 1 evaluation.
 Range (min … max):  3.125 ms … 9.479 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.538 ms             ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.647 ms ± 1.299 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▃ ▃▂▁▁▃▃▁▇▃▃▃▅▇▆▅▃▆▅▄▇▅▁▃▄▂▃▂▄█ ▁▄▁                   
  ▁▁▁▅██▇█████████████████████████████████▇▆▇▅▅▃▇▄▃▅▃▇▃▃▃▂▅ ▅
  3.12 ms        Histogram: frequency by time       8.67 ms <

 Memory estimate: 928 bytes, allocs estimate: 23.

In [31]:
d["Python (numpy)"] = minimum(b.times) / 1e6

3.124585

### hand-written

In [32]:
@pyexec """
global mysum

def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [33]:
mysum_py = @pyeval("mysum")

Python function: <function mysum at 0x7f47565f1f80>

In [34]:
x_py = pylist(x);

In [35]:
b = @benchmark $mysum_py($x_py)

BenchmarkTools.Trial: 25 samples with 1 evaluation.
 Range (min … max):  180.112 ms … 232.542 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     198.894 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   200.639 ms ±  15.324 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃ ▃         ▃ █      ▃▃                                        
  █▁█▁▁▁▁▇▁▇▁▁█▁█▁▁▁▁▁▁██▁▁▁▁▇▇▁▇▁▁▁▇▁▁▇▁▁▁▁▇▁▁▁▁▁▁▁▇▇▁▁▁▇▁▁▁▁▇ ▁
  180 ms           Histogram: frequency by time          233 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [36]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

180.112151

### built-in

In [37]:
# get the Python built-in "sum" function:
pysum = pybuiltins.sum

Python builtin_function_or_method: <built-in function sum>

In [38]:
b = @benchmark $pysum($x_py)

BenchmarkTools.Trial: 79 samples with 1 evaluation.
 Range (min … max):  60.487 ms … 72.896 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     63.272 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   63.880 ms ±  2.294 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

         █▁█▃▁▁▁█ ▃ █  ▃  ▁                                    
  ▄▁▁▄▁▇▄████████▇█▄█▇▄█▁▄█▇▇▁▁▄▇▄▁▁▁▇▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▄▁▄ ▁
  60.5 ms         Histogram: frequency by time        71.2 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [39]:
d["Python (built-in)"] = minimum(b.times) / 1e6

60.487306

## C

### hand-written

In [40]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [41]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib = tempname() * "." * Libdl.dlext

"/tmp/jl_7Mf8Xu5uDA.so"

In [42]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [43]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [44]:
c_sum(x) ≈ sum(x)

true

In [45]:
b = @benchmark c_sum($x)

BenchmarkTools.Trial: 544 samples with 1 evaluation.
 Range (min … max):  7.738 ms …  12.822 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.060 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.177 ms ± 895.236 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▁▆  ▄▂▁▄█▆▄▆▄▇▅▆▅▃▄▁▅▅▃▃ ▄▄▄▃▁▃▁▃   ▃▂                     
  ████▆▆█████████████████████████████▇▇▅██▆▆▆▄▃▆▇▇▃▃▆▄▁▁▄▃▁▃▄ ▆
  7.74 ms         Histogram: frequency by time        11.5 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [46]:
d["C"] = minimum(b.times) / 1e6

7.737621

### hand-written (with `-fast-math`)

In [47]:
const Clib_fastmath = tempname() * "." * Libdl.dlext

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $Clib_fastmath -`, "w") do f
    print(f, c_code)
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum_fastmath (generic function with 1 method)

In [48]:
b = @benchmark c_sum_fastmath($x)

BenchmarkTools.Trial: 700 samples with 1 evaluation.
 Range (min … max):  4.569 ms … 12.070 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.964 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.124 ms ±  1.492 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁ ▃▄       ▂▄█▅▄▃▁▅▃▄ ▃▂ ▁  ▃▄  ▂▃ ▃▂▂                     
  ▅█▇██▆▇▇▇█▇████████████████▇███▇▇██▇███▆▇▅▄▆▆▃▅▅▃▅▃▃▄▄▄▃▁▃ ▅
  4.57 ms        Histogram: frequency by time        10.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [49]:
d["C (fastmath)"] = minimum(b.times) / 1e6

4.569263

## Julia

### built-in

In [50]:
b = @benchmark sum($x)

BenchmarkTools.Trial: 990 samples with 1 evaluation.
 Range (min … max):  2.864 ms … 8.850 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.932 ms             ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.030 ms ± 1.190 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▂▂▃    ▂▁█▃█▂ ▂▁▂▂▄▃▃▂▆▂▂▂     ▃▂                     
  ▂▂▂████████████████████████████▇▇█▇▆████▇▄▇▄▅▆▆▄▄▅▂▄▅▂▃▂▄ ▄
  2.86 ms        Histogram: frequency by time       7.79 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [51]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

2.864477

### built-in (with `Vector{Any}`)

In [52]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

BenchmarkTools.Trial: 23 samples with 1 evaluation.
 Range (min … max):  197.502 ms … 229.688 ms  ┊ GC (min … max): 0.00% … 7.75%
 Time  (median):     219.898 ms               ┊ GC (median):    7.92%
 Time  (mean ± σ):   218.622 ms ±   7.501 ms  ┊ GC (mean ± σ):  5.37% ± 3.97%

  ▁               ▁      ▁  ▁    ▁▁ ▁ █  ▁ ▁▁█ ▁ █   ▁▁▁  ▁▁  ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁█▁▁▁▁██▁█▁█▁▁█▁███▁█▁█▁▁▁███▁▁██▁▁█ ▁
  198 ms           Histogram: frequency by time          230 ms <

 Memory estimate: 152.59 MiB, allocs estimate: 9999999.

In [53]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

197.502065

### hand-written

In [55]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [56]:
b = @benchmark mysum($x)

BenchmarkTools.Trial: 531 samples with 1 evaluation.
 Range (min … max):  7.781 ms …  12.627 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.287 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.402 ms ± 993.538 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▁ ▁ ▁▁▄▅▄▅▇█▄▆▇▄▅▃ ▁▃▁▂▂▄▃▁▅▄   ▃▄▄▄▁▁ ▁                    
  ██▆█▅██████████████▇██████████▄████████▅█▆▄▆▇▅▄▅▆▅▅▃▅▃▃▅▁▁▃ ▆
  7.78 ms         Histogram: frequency by time        11.9 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [57]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

7.780712

### hand-written (with `@fastmath`)

In [58]:
function mysum_fastmath(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @fastmath for a in A
        s += a
    end
    return s
end

mysum_fastmath (generic function with 1 method)

In [59]:
b = @benchmark mysum_fastmath($x)

BenchmarkTools.Trial: 1064 samples with 1 evaluation.
 Range (min … max):  2.592 ms … 8.253 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.476 ms             ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.676 ms ± 1.182 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁ ▃▁ ▁ ▃   ▃▇▇▅▇██▆▃▅▁▄ ▃   ▁ ▁▄    ▁                    
  ▃▄█▆██▆█▇█▅▇▆███████████████▇████████▇█▅▆▃▅▄▃▅▅▄▃▄▄▂▄▃▂▂▃ ▅
  2.59 ms        Histogram: frequency by time       7.55 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [60]:
d["Julia (hand-written, fastmath)"] = minimum(b.times) / 1e6

2.592236

## Summary

In [61]:
for (key, value) in sort(collect(d), by=x -> x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

Julia (hand-written, fastmath)......2.59
Julia (built-in)....................2.86
Python (numpy)......................3.12
C (fastmath)........................4.57
C...................................7.74
Julia (hand-written)................7.78
Python (built-in)..................60.49
Python (hand-written).............180.11
Julia (built-in, Any)..............197.5


And of course, our hand-written Julia implementation is type-generic!

In [62]:
mysum_fastmath(rand(ComplexF64, 10))

4.261091173761978 + 4.31994999246731im

# Supplement: What about other functions?

## Log

In [63]:
@which log(1.2)

log(x::Float64) in Base.Math at special/log.jl:267

In [64]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) = ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log(1.2)
@btime clog(1.2)
@btime llvmlog(1.2);

  1.262 ns (0 allocations: 0 bytes)
  5.050 ns (0 allocations: 0 bytes)
  2.264 ns (0 allocations: 0 bytes)


## Exp

In [67]:
@which exp(1.2)

exp(x::Union{Float16, Float32, Float64}) in Base.Math at special/exp.jl:326

In [68]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) = ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp(1.2);
@btime cexp(1.2);
@btime llvmexp(1.2);

  1.262 ns (0 allocations: 0 bytes)
  5.405 ns (0 allocations: 0 bytes)
  2.264 ns (0 allocations: 0 bytes)


## Matrix multiplication

In [69]:
N = 10
C = zeros(N, N);
A = rand(N, N);
B = rand(N, N);

In [70]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [71]:
C ≈ A * B

true

In [72]:
using BenchmarkTools

function mul_naive!(C, A, B)
    for m in axes(A, 1)
        for n in axes(B, 2)
            Cmn = zero(eltype(C))
            for k in axes(A, 2)
                @inbounds Cmn += A[m, k] * B[k, n]
            end
            C[m, n] = Cmn
        end
    end
end

mul_naive! (generic function with 1 method)

In [73]:
mul_naive!(C, A, B)
C ≈ A * B

true

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [74]:
using LoopVectorization

function mul_turbo!(C, A, B)
    @turbo for m in axes(A, 1)
        for n in axes(B, 2)
            Cmn = zero(eltype(C))
            for k in axes(A, 2)
                @inbounds Cmn += A[m, k] * B[k, n]
            end
            C[m, n] = Cmn
        end
    end
end

mul_turbo! (generic function with 1 method)

In [75]:
mul_turbo!(C, A, B)
C ≈ A * B

true

In [79]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [80]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib_gemm = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib_gemm -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64}, A::Array{Float64}, B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A, 1)::Clong, size(A, 2)::Clong, size(B, 2)::Clong)::Cvoid

c_gemm (generic function with 1 method)

In [81]:
c_gemm(C, A, B)
C ≈ A * B

true

In [82]:
@btime mul_naive!($C, $A, $B);
@btime mul_turbo!($C, $A, $B);
@btime mul!($C, $A, $B);
@btime c_gemm($C, $A, $B)

  477.718 ns (0 allocations: 0 bytes)
  178.391 ns (0 allocations: 0 bytes)
  341.330 ns (0 allocations: 0 bytes)
  261.449 ns (0 allocations: 0 bytes)


**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` can be slower than `mul!`.